In [6]:
import random
import string

# Функция для проверки числа на простоту
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

# Функция для генерации простых чисел заданной длины
def generate_large_prime(keysize=1024):
    while True:
        num = random.getrandbits(keysize)
        if is_prime(num):
            return num

# Функция для вычисления НОД
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# Функция для вычисления обратного по модулю
def multiplicative_inverse(e, phi):
    d, x1, x2, y1 = 0, 0, 1, 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi, e = e, temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2, x1 = x1, x
        d, y1 = y1, y
    
    if temp_phi == 1:
        return d + phi

# Функция для генерации ключей
def generate_keypair(keysize):
    p = generate_large_prime(keysize)
    q = generate_large_prime(keysize)
    n = p * q
    phi = (p-1) * (q-1)

    e = random.randrange(1, phi)
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    d = multiplicative_inverse(e, phi)
    
    return ((e, n), (d, n))

# Функция для быстрого возведения в степень по модулю
def fast_modular_exponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

# Функция для шифрования
def encrypt(pk, plaintext):
    key, n = pk
    cipher = [fast_modular_exponentiation(ord(char), key, n) for char in plaintext]
    return cipher

# Функция для дешифрования
def decrypt(pk, ciphertext):
    key, n = pk
    plain = [chr(fast_modular_exponentiation(char, key, n)) for char in ciphertext]
    return ''.join(plain)

# Функция для генерации случайных сообщений заданной длины
def generate_random_message(length):
    letters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(letters) for i in range(length))

# Функция для тестирования алгоритма RSA
def test_rsa(keysize, num_tests):
    public, private = generate_keypair(keysize)
    
    for i in range(num_tests):
        message = generate_random_message(random.randint(1, 100))
        encrypted_msg = encrypt(public, message)
        decrypted_msg = decrypt(private, encrypted_msg)
        
        if message != decrypted_msg:
            print(f"Тест {i+1} провален.")
            print(f"Оригинальное сообщение: {message}")
            print(f"Расшифрованное сообщение: {decrypted_msg}")
            return
        else:
            print(f"Тест {i+1} пройден успешно.")
    
    print(f"Все {num_tests} тестов пройдены успешно.")

# Вызов функции тестирования
if __name__ == '__main__':
    test_rsa(1024, 10)
